In [25]:
import pandas as pd
import re
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [26]:
# load data from csv file (cleaned dataset)

df_4 = pd.read_csv("C:/Users/Amin/project-amin/data cleaning (eda)/sephoramy_webscrape_dataset_clean.csv")

In [27]:
df_4.head()

,Category,URL,brand,name,price,skin_concern,rating,Combination,Dry,Normal,Oily,Sensitive
0,cleanser-and-exfoliator,https://www.sephora.my/products/origins-go-go-...,ORIGINS,"Go Go Ginzing™ Skincare Trio To Cleanse, Hydra...",130,"Dark Circles, Dryness, Dullness, Puffiness",0.0,1,1,1,1,0
1,cleanser-and-exfoliator,https://www.sephora.my/products/gallinee-face-...,GALLINÉE,Face Mask & Scrub,45,"Dryness, Dullness, Uneven Skin Tone",4.5,1,1,1,1,1
2,cleanser-and-exfoliator,https://www.sephora.my/products/the-inkey-list...,THE INKEY LIST,Oat Cleansing Balm • 150ml,46,Balm,4.0,1,1,1,1,0
3,cleanser-and-exfoliator,https://www.sephora.my/products/fresh-soy-face...,FRESH,Soy Face Cleanser,177,"Dryness, Ageing, Dullness, Firmness & Elasticity",4.5,1,1,1,1,1
4,cleanser-and-exfoliator,https://www.sephora.my/products/origins-best-o...,ORIGINS,"Best Of The Season Skincare Set To Cleanse, Pr...",65,"Dryness, Dullness, Oiliness",3.5,1,1,1,1,0


In [37]:
# recommend product by user features
category = df_4.Category.unique().tolist()
skintype = df_4.columns[7:].tolist()

def recommend_products_by_user(category, skintype):
    df = df_4[df_4['Category'] == category][df_4[skintype] == 1]
    df = df.sort_values('rating', ascending=False).head()
    
    print('These are top products related to your features:')
    return df

In [38]:
recommend_products_by_user('toner', 'Sensitive')

These are top products related to your features:


<ipython-input-37-9b565c0b4224>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df_4[df_4['Category'] == category][df_4[skintype] == 1]


,Category,URL,brand,name,price,skin_concern,rating,Combination,Dry,Normal,Oily,Sensitive
148,toner,https://www.sephora.my/products/laneige-fresh-...,LANEIGE,Fresh Calming Balancing Toner,120,Skin Concerns: Dryness,4.5,1,1,1,1,1
120,toner,https://www.sephora.my/products/indie-lee-coq-...,INDIE LEE,CoQ-10 Toner,143,"Dryness, Dullness, Firmness & Elasticity, Unev...",4.5,1,1,1,1,1
135,toner,https://www.sephora.my/products/tatcha-the-ess...,TATCHA,The Essence • 150ml,410,"Dryness, Dullness, Fine Lines & Wrinkles, Unev...",4.5,1,1,1,1,1
155,toner,https://www.sephora.my/products/glamglow-super...,GLAMGLOW,SUPERTONER ™ Exfoliating Pore Clarifying Acid ...,200,"Dullness, Oiliness, Visible Pores",4.5,1,1,1,1,1
129,toner,https://www.sephora.my/products/dior-capture-t...,DIOR,Capture Totale DreamSkin Care & Perfect Emulsion,386,"Dryness, Dullness, Pigmentation & Dark Spots, ...",4.5,1,1,1,1,1


In [47]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df_4['skin_concern'])

In [48]:
test = pd.DataFrame(tfidf_matrix.toarray(), index=df_4.name, columns=tf.get_feature_names())
test.head()

,acne,acne blemishes,ageing,ageing dark,ageing dryness,ageing dullness,ageing fine,ageing pigmentation,ageing uneven,balm,...,uneven,uneven skin,visible,visible pores,wrinkles,wrinkles firmness,wrinkles oiliness,wrinkles pigmentation,wrinkles uneven,wrinkles visible
name,,,,,,,,,,,,,,,,,,,,,
"Go Go Ginzing™ Skincare Trio To Cleanse, Hydrate & Brighten (Limited Edition)",0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Face Mask & Scrub,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.30555,0.30555,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Oat Cleansing Balm • 150ml,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,...,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Soy Face Cleanser,0.0,0.0,0.249186,0.0,0.0,0.391504,0.0,0.0,0.0,0.0,...,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Best Of The Season Skincare Set To Cleanse, Prep & Hydrate (Limited Edition)",0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

indices = pd.Series(df_4.index, index=df_4['name'])

In [57]:
def recommend_products_by_content(product):
    idx = indices[product]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:5]
    product_indices = [i[0] for i in sim_scores]
    return df_4.iloc[product_indices]

In [58]:
recommend_products_by_content('Fresh Calming Balancing Toner')

,Category,URL,brand,name,price,skin_concern,rating,Combination,Dry,Normal,Oily,Sensitive
148,toner,https://www.sephora.my/products/laneige-fresh-...,LANEIGE,Fresh Calming Balancing Toner,120,Skin Concerns: Dryness,4.5,1,1,1,1,1
165,toner,https://www.sephora.my/products/erno-laszlo-li...,ERNO LASZLO,Light Controlling Lotion • 200ml,440,"Skin Concerns: Dryness, Dullness, Oiliness, Vi...",4.5,1,0,1,1,1
201,moisturiser,https://www.sephora.my/products/the-inkey-list...,THE INKEY LIST,Peptide Moisturizer • 50ml,71,Skin Concerns: Ageing,4.0,1,1,1,1,0
30,cleanser-and-exfoliator,https://www.sephora.my/products/origins-detox-...,ORIGINS,"Detox With Delight Cleansing, Purifying & Spot...",95,"Skin Concerns: Acne & Blemishes, Dullness, Oil...",0.0,1,1,1,1,0
7,cleanser-and-exfoliator,https://www.sephora.my/products/laneige-moist-...,LANEIGE,Moist Cream Cleanser,100,Dryness,4.5,1,1,1,1,0
